In [13]:
from elasticsearch import Elasticsearch

In [14]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [15]:
# 参考:GitHubProjects\MLBase\H_BasicTheory\机器学习\NLP\bm25
body = {
    "settings": {
        "similarity": {
            "custom_similarity": {
                "type": "BM25",  # 默认为Okapi_BM25 
                "k1": 0.1,  # 默认1.2
                "b": 0.1  # 默认0.75
            }
        }
    },
    "mappings": {
        "properties": {
            "information": {
                "type": "text",
                "similarity": "custom_similarity"
            }
        }
    }
}

if es.indices.exists('es0'):
    es.indices.delete('es0')
es.indices.create('es0', body=body)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'es0'}

In [16]:
es.index(index='es0', id=0, body={"information": "屈光不正"})
es.index(index='es0', id=1, body={"information": "左眼屈光不正"})
es.index(index='es0', id=2, body={"information": "左眼屈光不正、右眼屈光不正、双眼屈光不正"})
es.index(index='es0', id=3, body={"information": "脾全切术后"})
es.index(index='es0', id=4, body={"information": "阑尾切除术后"})
es.index(index='es0', id=5, body={"information": "宫颈切除术后"})
es.index(index='es0', id=6, body={"information": "甲状腺切除术后"})
es.index(index='es0', id=7, body={"information": "屈光参差"})

{'_index': 'es0',
 '_type': '_doc',
 '_id': '7',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 7,
 '_primary_term': 1}

In [18]:
es.search(index='es0', body={
    "query": {
        "match": {
            "information": {
                "query": "屈光不正"
            }
        }
    }
})

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 4, 'relation': 'eq'},
  'max_score': 3.4689374,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': '2',
    '_score': 3.4689374,
    '_source': {'information': '左眼屈光不正、右眼屈光不正、双眼屈光不正'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '0',
    '_score': 3.288028,
    '_source': {'information': '屈光不正'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '1',
    '_score': 3.2794766,
    '_source': {'information': '左眼屈光不正'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '7',
    '_score': 1.3917166,
    '_source': {'information': '屈光参差'}}]}}

### query="屈光不正"时不同参数_score测试结果

|               | 屈光不正  | 左眼屈光不正 | 左眼屈光不正、右眼屈光不正、双眼屈光不正 | 屈光参差  |
| ------------- | --------- | ------------ | ---------------------------------------- | --------- |
| k1=1.2,b=0.75 | 3.9715233 | 3.478507     | 3.8502555                                | 1.6810182 |
| k1=1.2,b=1    | 4.274436  | 3.5519967    | 3.5519962                                | 1.8092315 |
| k1=1.2,b=0    | 3.2752175 | 3.2752175    | 5.1467705                                | 1.3862942 |
| k1=3,b=0.75   | 4.315581  | 3.5614014    | 4.121621                                 | 1.8266468 |
| k1=0,b=0.75   | 3.2752175 | 3.2752175    | 3.2752175                                | 1.3862944 |
| k1=3,b=0      | 3.2752175 | 3.2752175    | 6.550435                                 | 1.3862944 |
| k1=0,b=1      | 3.2752175 | 3.2752175    | 3.2752175                                | 1.3862944 |